In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

### Netflix Prize dataset import:

In [3]:
os.listdir(os.path.join('.','netflix_dataset'))

['combined_data_1.txt',
 'combined_data_2.txt',
 'combined_data_3.txt',
 'combined_data_4.txt',
 'movie_titles.csv',
 'probe.txt',
 'qualifying.txt',
 'README']

In [4]:
# read the CSV file line by line
with open(os.path.join('.', 'netflix_dataset','movie_titles.csv'), 'r', encoding='latin-1') as file:
    lines = file.readlines()

# process each line
data = []
for line in lines:
    parts = line.strip().split(',', 2)  # split the line using first two commas only, to ensure only id, year and title are catched, as they cover two commas
    if len(parts) == 3:  # this way it is checked whether its actually three parts
        # concatenate the result and append it
        data.append((parts[0],parts[1], parts[2]))

# create a dataframe where the processed csv file will be stored in
movie_title_df = pd.DataFrame(data, columns=['movieId','year','title'])

# convert id column to integer for merging later
movie_title_df['movieId'] = movie_title_df['movieId'].astype(int)

In [5]:
# get the complete list of files in the netflix directory
netflix_files = os.listdir(os.path.join('.', 'netflix_dataset'))

# make a filter which only contains the combined datafiles
combined_files = [file for file in netflix_files if 'combined' in file]

# define a generator expression to yield each file's data and make a pd dataframe where only the combined files are selected
data_generator = (pd.read_csv(os.path.join('.', 'netflix_dataset', file), sep=',', header=None, names=['userId', 'rating', 'date']) for file in combined_files)

# concatenate the result
netflix_df = pd.concat(data_generator, ignore_index=True)

### MovieLens data import:

In [6]:
# import each csv
df_links = pd.read_csv('movielens_dataset2/links.csv',sep=',')
df_movies = pd.read_csv('movielens_dataset2/movies.csv',sep=',')
df_ratings = pd.read_csv('movielens_dataset2/ratings.csv',sep=',')
df_tags = pd.read_csv('movielens_dataset2/tags.csv',sep=',')

In [7]:
# look at each dataset individually
df_links
df_movies
df_ratings
df_tags

# by looking at the dataframes, they can be merged into two ones instead of four to create a user and movie dataframe
movies_df = pd.merge(df_links,df_movies,on='movieId',how='outer')
# merge by userId and movieId to rating, timestamp of rating, tag and timestmap of tag per user and the movie the review has been givent to
users_df = pd.merge(df_ratings,df_tags,on=['userId','movieId'],how='outer')
users_df
users_df.isnull().sum()
df_tags['userId'].nunique()
df_ratings['userId'].nunique()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
86532,288967,14418234,845861.0
86533,288971,11162178,878958.0
86534,288975,70199,150392.0
86535,288977,23050520,1102551.0


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
86532,288967,State of Siege: Temple Attack (2021),Action|Drama
86533,288971,Ouija Japan (2021),Action|Horror
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),Documentary
86535,288977,Skinford: Death Sentence (2023),Crime|Thriller


,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119
...,...,...,...,...
33832157,330975,8340,2.0,1091583256
33832158,330975,8493,2.5,1091585709
33832159,330975,8622,4.0,1091581777
33832160,330975,8665,3.0,1091581765


,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746
...,...,...,...,...
2328310,330923,176599,politically correct,1507547491
2328311,330933,3317,coming of age,1351279384
2328312,330933,3317,sexuality,1351279389
2328313,330947,5782,Not Luc Besson,1154110902


,userId,movieId,rating,timestamp_x,tag,timestamp_y
0,1,1,4.0,1.225735e+09,NaN,NaN
1,1,110,4.0,1.225865e+09,NaN,NaN
2,1,158,4.0,1.225734e+09,NaN,NaN
3,1,260,4.5,1.225735e+09,NaN,NaN
4,1,356,5.0,1.225735e+09,NaN,NaN
...,...,...,...,...,...,...
35827120,330975,8340,2.0,1.091583e+09,NaN,NaN
35827121,330975,8493,2.5,1.091586e+09,NaN,NaN
35827122,330975,8622,4.0,1.091582e+09,NaN,NaN
35827123,330975,8665,3.0,1.091582e+09,NaN,NaN


userId                0
movieId               0
rating           599023
timestamp_x      599023
tag            33498827
timestamp_y    33498810
dtype: int64

25280

330975

In **tag** and **timestamp_y** form **users_df** column are lots of null values, meaning that not that many users have given a tag to their review. Comparing the amount of unique users in the tags csv compared to the ratings df, it is normal to have that many null values after merging.

### Perform cleaning:

In [8]:
# extract number between brackets for creating release year column with a regex pattern which corresponds to four digits between brackets
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)')

# cut off the last six characters to clean up the movie titles
movies_df['title'] = movies_df['title'].str[:-6]

# show cleaned result
movies_df

,movieId,imdbId,tmdbId,title,genres,year
0,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,113497,8844.0,Jumanji,Adventure|Children|Fantasy,1995
2,3,113228,15602.0,Grumpier Old Men,Comedy|Romance,1995
3,4,114885,31357.0,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,113041,11862.0,Father of the Bride Part II,Comedy,1995
...,...,...,...,...,...,...
86532,288967,14418234,845861.0,State of Siege: Temple Attack,Action|Drama,2021
86533,288971,11162178,878958.0,Ouija Japan,Action|Horror,2021
86534,288975,70199,150392.0,The Men Who Made the Movies: Howard Hawks,Documentary,1973
86535,288977,23050520,1102551.0,Skinford: Death Sentence,Crime|Thriller,2023


In [9]:
# convert to a readable datetime format
users_df['date'] = pd.to_datetime(users_df['timestamp_x'],unit='s')
users_df['timestamp_tag'] = pd.to_datetime(users_df['timestamp_y'],unit='s')

# convert to day format and strip off time
users_df['date'] = users_df['date'].dt.date
users_df['timestamp_tag'] = users_df['timestamp_tag'].dt.date

# drop original columns
users_df = users_df.drop(['timestamp_x','timestamp_y'],axis=1)

Convert everything to parquet for performance purposes:

In [10]:
users_df.to_parquet('parquets/users_df')
movies_df.to_parquet('parquets/movies_df')
movie_title_df.to_parquet('parquets/movie_title_df')
netflix_df.to_parquet('parquets/netflix_df')